# Estruturas Mistas de Aço e Concreto

In [1]:
import handcalcs.render
import forallpeople
from math import pi
from IPython.display import display, Markdown, Math
import sympy
import csv
import ipywidgets as widgets

forallpeople.environment('structural', top_level = True)
%decimal_separator ,


# 0. Entrada de dados - geometria e materiais

## 0.1 Materiais

### __Propriedades do Aço__

Mudanças a fazer:
* Incluir outros arquivos CSV com as séries de perfil que interessam (UB, ASCI, IPE, I rolado)
* Incluir drop-down para escolher a série desejada
* Conforme a série escolhida, abrir o arquivo certo, e obter os dados
* Relacionar as variáveis do notebook (com unidades) com as variáveis obtidas no dicionario do perfil.

In [2]:
%%render params 2
# Aço estrutural:
f_yd = 355 * N/mm**2 # $γ_a=1$
E_a = 210 * kN/mm**2

# Aço do deck
f_yp= 350 * N/mm**2 #$γ_p=1$

# Aço dos conectores
f_uk = 500 * N/mm**2
gamma_v=1.25
f_ud=400 / gamma_v * N/mm**2

<IPython.core.display.Latex object>

### __Propriedades do Concreto__

In [3]:
%%render params 2

# Concreto:
f_ck = 25 * MPa
gamma_c = 1.5
f_cd = f_ck/gamma_c
E_cm = 20.7 * kN/mm**2

# Razão modular
n_0 = E_a/E_cm # razão modular


<IPython.core.display.Latex object>

## 0.2 Geometria

### __Geometria da laje composta__

In [4]:
%%render params
h_t = 150 * mm # espessura total da laje composta
h_r = 70 * mm # altura da nervura (rib)
h_c = h_t - h_r

<IPython.core.display.Latex object>

### Conectores

In [5]:
%%render params 2

# Conectores

d = 19 * mm # diâmetro do conector


<IPython.core.display.Latex object>

### __Viga de aço (até agora somente perfil simétrico)__

In [6]:
# Seletor de perfil e gerador do dicionário com seus dados
with open('VS_NBR.csv', 'r', encoding='utf-8-sig') as csv_file2:
    csv_reader2 = csv.reader(csv_file2)
    #shape_list=[]
    shape_name_list=[]
    
    for i in csv_reader2:
        #shape_list.append(i)
        shape_name_list.append(i[0]) # Cria lista com os nomes das seções
        #print(i)
    shape_name_list.pop(0)

shape_selector=widgets.Dropdown(
    options=shape_name_list,
    #value='2',
    description='Viga de aço:',
    disabled=False,
)

display(shape_selector)

Dropdown(description='Viga de aço:', options=('VS 150x15', 'VS 150x18', 'VS 150x20', 'VS 150x19', 'VS 150x21',…

In [22]:
selected_shape=shape_selector.value

with open('VS_NBR.csv', 'r', encoding='utf-8-sig') as csv_file:
    csv_reader = csv.DictReader(csv_file)
        
    fun = lambda dic: True if dic.get('Section')==selected_shape else False # Mudar a string 'VS...' para uma variável que pega o valor escolhido num dropdown
    filtered = filter(fun, csv_reader)
    #shape_dic = list(filter(fun, csv_reader))
    #print(shape_dic)
    shape_dic=(next(filtered))
    
    # Este bloco converte os valores numéricos de str para float
    itervals = iter(shape_dic.keys())
    next(itervals) # para pular o primeiro, que é str (o nome da seção)
    for n in itervals:
            shape_dic[n] = float(shape_dic[n])
            
    print(shape_dic)

{'Section': 'VS 200x22', 'h[mm]': 200.0, 'b[mm]': 120.0, 'tw[mm]': 4.8, 'tf[mm]': 8.0, 'd[mm]': 184.0, 'k[mm]': 5.0, 'A[cm2]': 27.9, 'Iy[cm4]': 2017.0, 'Iz[cm4]': 231.0, 'Ip[cm4]': 2248.0, 'iy[mm]': 85.0, 'iz[mm]': 28.7, 'ip[mm]': 89.7, 'max Sy[cm3]': 29.5, 'max Sz[cm3]': 14.39, 'Wy[cm3]': 202.0, 'Wz[cm3]': 38.0, 'Ay[cm2]': 16.14, 'Az[cm2]': 8.77, 'It[cm4]': 4.8, 'max ω[cm2]': 57.6, 'Iω[cm6]': 21234.0, 'iω[mm]': 30.7, 'Wω[cm4]': 368.65, 'max Sω[cm4]': 138.24, 'Wpl,y[cm3]': 59.0, 'Wpl,z[cm3]': 225.0, 'Wpl,ω[cm4]': 552.96, 'αpl,y': 0.292, 'αpl,z': 5.921, 'αpl,ω': 1.5, 'Apl,y[cm2]': 19.2, 'Apl,z[cm2]': 9.12, 'Npl[kN]': 656.59, 'Vpl,y[kN]': 260.5, 'Vpl,z[kN]': 123.738, 'Mpl,y[kNm]': 52.76, 'Mpl,z[kNm]': 13.78, 'G[kg/m]': 21.9, 'Am[m2/m]': 0.871, 'V[cm3/m]': 2790.0, 'Am/V[1/m]': 312.007}


In [23]:
%%render 2 params
A = shape_dic['A[cm2]'] * (10*mm)**2
A_s = 7650 * mm**2
d = 406 * mm
d_sup = d/2
d_inf = d/2
t_f = 12.8 * mm
b_f = 178 * mm
t_w = 7.8 * mm
h_w = d - 2*t_f### Resistência do conector 

<IPython.core.display.Latex object>

In [9]:
%%render 2
# Resistência do aço do conector
P_Rd1 = (0.8 * f_uk * ((pi * d**2) / 4)) / gamma_v

# Resistência ao esmagamento local do concreto
P_Rd2 = (0.29 * d**2 * (f_ck * E_cm )**(0.5)) / gamma_v

P_Rd = min(P_Rd1, P_Rd2)

<IPython.core.display.Latex object>

## 1. Definição de ações e carregamentos correspondentes

### 1.1 Ações permanentes

In [10]:
%%render 2 symbol
# Aço estrutural
delta_a = 77 * kN/m**3 # aço

# Concreto leve LC25/28
delta_LC = 19.5 * kN/m**3 # peso específico do concreto da laje
V_conc = 0.125 * m**3 / m **2 # m³ de concreto por m² de piso


<IPython.core.display.Latex object>

### 1.2 Cargas permanentes por m²

In [11]:
%%render 2
g_kp_A = 0.1 * kN / m**2 # peso do deck
g_kc_A = V_conc * delta_LC #peso de concreto
g_k_LAJE_A = g_kp_A + g_kc_A # peso da laje composta
g_H2OA = 0.125 * kN/m**3 # acréscimo de 1kN/m² para o concreto fresco
g_k2_A = 1.3 * kN/m**2 # acabamentos - piso e forros

<IPython.core.display.Latex object>

### 1.3 Cargas variáveis por m²
Andares na categoria C3 do EN 1991-1-1.

In [12]:
%%render 2 symbol
q_k1 = 5 * kN/m**2 # pessoas
q_k2 = 1.2 * kN/m**2 # partições móveis

<IPython.core.display.Latex object>

### 1.4 Cargas características na viga

In [13]:
%%render
d_v = 4 * m # distância entre vigas

<IPython.core.display.Latex object>

In [14]:
%%render 2
g_k1 = g_k_LAJE_A * d_v # peso da laje composta sobre a viga de aço
g_k2 = g_k2_A * d_v # peso dos acabamentos na viga composta
g_k_FIRE = 2.2 * kN/m # estimativa do peso da viga com proteção contra incêndio
q_k = (q_k1+q_k2)*d_v # carga variável para a viga composta

<IPython.core.display.Latex object>

### 1.5 Combinação de projeto para viga no ELU de ruptura à flexão e $M_{Ed}$

In [15]:
%%render 1
g_d = 1.35 * (g_k1 + g_k2+ g_k_FIRE)
q_d = 1.5 * (q_k)

# O vão é igual a:
#   Dist. entre colunas
# - altura dos perfis das colunas (20cm/2 cada lado)
# - dis. pt. ef. de suporte (10cm)
L = 9 * m - 4 * 0.1 * m
M_Ed = ((g_d + q_d) *  L**2)/8

<IPython.core.display.Latex object>

### 1.6 Escolha do perfil: 406X178 UB 60 - PERFIL SIMÉTRICO

In [16]:
%%render 2 params
A_s = 7650 * mm**2
d = 406 * mm
d_sup = d/2
d_inf = d/2
t_f = 12.8 * mm
#t_f = 25.4 * mm
b_f = 178 * mm
#b_f = 300 * mm
t_w = 7.8 * mm
h_w = d - 2*t_f

<IPython.core.display.Latex object>

## 2. Largura efetiva $b_{ef}$ da aba de concreto

In [17]:
%%render 2
b_ef = L/4 + 0.1 # 10cm é a distância entre conectores


<IPython.core.display.Latex object>

## 3. Cálculo para interação total

### 3.1 Cálculo de $M_{pl,Rd}$

In [18]:
%%render 2 
alpha_c = 0.85
N_cf = b_ef * h_c * (alpha_c * f_cd) 
N_pla = f_yd * A_s
rho_w=1 # Este valor depende de $V_{Ed}$ ver pg. 158 pdf Hanswille
N_plw = f_yd * (1 - rho_w) * t_w * h_w

<IPython.core.display.Latex object>

In [19]:
def LNP_position(N_cf, N_pla):
    """
    Define a posição da linha neutra
    """
    # Condição para LNP no perfil
    if N_cf < N_pla:
        display(Markdown("$N_{c,f} < N_{a,pl}$: O aço resiste mais força normal, então toda a aba de concreto está comprimida no ELU, logo a LNP está no perfil de aço."))
        M_plRd=LNP_steel(N_pla, N_cf, b_ef, t_f, f_yd)
        return M_plRd
    # Condição para a LNP na aba superior de concreto
    elif N_pla < N_cf:
        display(Markdown("$N_{c,f} > N_{a,pl}$: O concreto resiste mais força normal, então todo o perfil está tracionado, e a LNP está na aba de concreto."))
        # COLOCAR AQUI O RESTO DOS CALCULOS PRA DETERMINAR MplRd
        M_plRd=MplRd_calc('A')
        return M_plRd
        
def LNP_steel(N_pla, N_cf, b_f, t_f, f_yd):
    """
    Calcula a posição da LNP se estiver no perfil de aço
    """
    N_ac = N_pla - N_cf
    if N_ac <= 2 * b_f * t_f * f_yd:
        display(Markdown("LNP na aba superior da viga de aço"))
        #Seguindo formulação do Hanswille pg. 73
        M_plRd=MplRd_calc('B')
        return M_plRd
    elif N_ac > 2 * b_ef * t_f * f_yd:
        display(Markdown("LNP na alma da viga de aço"))
        M_plRd=MplRd_calc('C')
        return M_plRd
        
def MplRd_calc(x):
    """
    Calcula o momento plástico resistente, conforme o caso para a LNP
    """
    z_a = h_t + d_sup # dist da borda sup. ao CG da seção de aço
    z_w = h_t + t_f + h_w * 0.5 # dist. da borda sup. ao CG da alma
    N_f = 2 * f_yd * b_f * t_f
    if x == 'A':
        z_pl = (N_pla - N_plw)/(alpha_c * f_cd * b_ef)
        print(f"z_pl={z_pl}")
        M_plRd = N_pla * (z_a - 0.5 * z_pl) - N_plw * (z_w - 0.5 * z_pl)
        return M_plRd
    if x == 'B':
        z_pl = h_t + (N_pla - N_plw - N_cf)/(2 * f_yd * b_f)
        M_plRd = N_pla * (z_a - (h_t - h_c)*0.5) - N_plw * (z_w - (h_t - h_c)*0.5) - N_f*(z_pl-h_t)*(z_pl+h_c)/(2*t_f)
        return M_plRd
    if x == 'C':
        z_pl = h_t + t_f + (N_pla - N_plw - N_cf - N_f)/(2 * f_yd * rho_w * t_w)
        N_w = 2 * f_yd * t_w * (z_pl - h_t - t_f)
        M_plRd = N_pla * (z_a - (h_t - h_c)*0.5) - N_plw * (z_w - (h_t - h_c)*0.5) - N_f * (t_f + h_t + h_c) * 0.5 - N_w * (z_pl + t_f + h_c)*0.5
        return M_plRd

In [20]:
M_plRd=LNP_position(N_cf, N_pla)

$N_{c,f} < N_{a,pl}$: O aço resiste mais força normal, então toda a aba de concreto está comprimida no ELU, logo a LNP está no perfil de aço.

LNP na aba superior da viga de aço

In [21]:
%%render 1

M_plRd

<IPython.core.display.Latex object>

### Registro de testes e notas para melhorar entrada de dados

* Dados de perfil poderiam ser obtidos de um dicionário, para facilitar a escolha a partir de um CSV.
* Adicionar um recurso de plotagem para desenhar a seção e as tensões no ELU.